# Pandas 와 Plotly를 이용한 Stochastic 차트 그리기

Stochastic 은 주식 분석에서 MACD 와 같이 기술적 분석에 사용되는 보조지표로써, 공식 명칭은 Stochastic Oscillator 이다.
Stochastic 은 현재주가가 일정한 기간의 주가 변동폭중 어디에 위치하는지를 백분율로 나타낸 지표이다.

- Stochastic 은 최근 n일 간의 최고가와 최저가의 범위 내에서 혀재가격의 위치를 포시할 때 매수세가 매도세 보다 가앟ㄹ 떄는 그위치가 높게 형성되고, 매도세가 매수세 보다 강할 때는 그위치가 낮게 형성된다는 것을 이용한 것이다.예를들어, 최근 5일간 최고가가 15,000원이고 최저가가 10,000원인 주식이 있을때, 현재가가 14,000원이라면 매수세가 강하여 오르는 추세임을 알 수 있다.  만일 현재가가 11,000원이라면 매도세가 강하여 내리는 추세임을 알 수 있다.

- Fast%K (K): n(15)일 동안의 최고가(high)와 최저가(low) 사이 중 현재 종가(close)의 상대적 위치를 판단하는 값 
가격이 지속적으로 상승하고 있다면 Stochastic 값은 100에 가까워 지며, 반대로 지속적으로 하락하고 있다면 Stochastic 값은 0에 가까워 지는 경향을 나타낸다.

- Slow%K (D): Fast%D라고도 하며 m(5)일간의 Fast%K의 이동평균 값
기본값으로 5일을 설정하며 Fast%K 값을 일반화하는 역할을 한다.

- Slow%D (J): t(3)일간의 Slow%K의 이동평균 값
기본값으로 3일을 설정하며 Slow%K 값을 일반화 하는 역할을 한다.


Stochastic 을 이용한 주식분석 : 

Stochastic을 이용해 주식분석을 할때는 Fast%K보다는 Slow%K와 Slow%D를 주로 이용한다. 그 이유는 실제 주식시장에서의 Fast%K는 변화가 잦고 속임수가 많아 정확하지 않기 때문이다.

- Slow%K선이 20값을 상향돌파 할 경우: Slow%K값이 20아래에 있다는 것은 지속적인 하락으로 인한 초과매도를 의미한다. ​ 매수포지션(과매도로 인한 상승 전환)


- Slow%K선이 80값을 하향돌파 할 경우: Slow%K값이 80위에 있다는 것은 지속적인 상승으로 인한 초과매수를 의미한다. ​매도포지션(과매수로 인한 하락 전환)

- Slow%K선이 Slow%D선을 상향돌파 할 경우: 매수 포지션

- Slow%K선이 Slow%D선을 하향돌파할 경우: 매도 포지션


In [3]:
import pandas as pd

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)

code_df = code_df[['회사명','종목코드']]

code_df = code_df.rename(columns={'회사명':'name','종목코드':'code'})
code_df.head()

,name,code
0,GS글로벌,001250
1,HSD엔진,082740
2,KG케미칼,001390
3,LG이노텍,011070
4,OCI,010060


In [10]:
#일자별 주식 데이터 가져오기

def get_url(item_name, code_df) :
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False)
    
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
    
    print("요청 URL ={}".format(url))
    return url

item_name= '신라젠'
url = get_url(item_name,code_df)

df = pd.DataFrame()

for page in range(1, 21):

    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
 
df = df.dropna()
df.head()

요청 URL =http://finance.naver.com/item/sise_day.nhn?code=215600


,날짜,종가,전일비,시가,고가,저가,거래량
1,2019.06.21,56800.0,900.0,58000.0,58300.0,56500.0,332732.0
2,2019.06.20,57700.0,1700.0,56100.0,57900.0,56100.0,487092.0
3,2019.06.19,56000.0,600.0,55900.0,56400.0,55200.0,331231.0
4,2019.06.18,55400.0,1300.0,57000.0,57400.0,55000.0,495082.0
5,2019.06.17,56700.0,100.0,57500.0,58500.0,56600.0,421203.0


In [11]:
# 한글로 된 컬럼명을 영어로 바꿔줌
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', 
                         '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})
 
# 데이터의 타입을 int형으로 바꿔줌
df[['close', 'diff', 'open', 'high', 'low', 'volume']] \
    = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
 
# 컬럼명 'date'의 타입을 date로 바꿔줌
df['date'] = pd.to_datetime(df['date'])
 
# 일자(date)를 기준으로 오름차순 정렬
df = df.sort_values(by=['date'], ascending=True)
 
# 상위 5개 데이터 확인
df.head()


,date,close,diff,open,high,low,volume
298,2018-08-27,66300,1300,66200,67500,64700,1302620
297,2018-08-28,67400,1100,67300,69100,66500,1677504
296,2018-08-29,66100,1300,67900,68100,65300,1044419
295,2018-08-30,68500,2400,65800,68600,65400,1002923
294,2018-08-31,76400,7900,67900,79100,67700,5987954


In [12]:
# 일자(n,m,t)에 따른 Stochastic(KDJ)의 값을 구하기 위해 함수형태로 만듬 
def get_stochastic(df, n=15, m=5, t=3):
    
    # 입력받은 값이 dataframe이라는 것을 정의해줌
    df = pd.DataFrame(df)
    
    # n일중 최고가
    ndays_high = df.high.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df


In [13]:
df = get_stochastic(df)
df.head()

,date,close,diff,open,high,low,volume,kdj_d,kdj_j,kdj_k
298,2018-08-27,66300,1300,66200,67500,64700,1302620,57.142857,57.142857,57.142857
297,2018-08-28,67400,1100,67300,69100,66500,1677504,59.675325,58.831169,61.363636
296,2018-08-29,66100,1300,67900,68100,65300,1044419,46.479836,51.773264,31.818182
295,2018-08-30,68500,2400,65800,68600,65400,1002923,63.046953,57.785898,86.363636
294,2018-08-31,76400,7900,67900,79100,67700,5987954,70.034853,64.107939,81.250000


In [16]:
def get_macd(df, short=12, long=26, t=9) :
    
    # 입력받은 값이 dataframe 이라는 것을 정의해줌
    
    df = pd.DataFrame(df)
    
    # MACD 관력 수식
    ma_12 = df.close.ewm(span=12).mean() #EMA(지수이동평균)
    ma_26 = df.close.ewm(span=26).mean()
    
    macd = ma_12 - ma_26 # MACD
    
    macds = macd.ewm(span=9).mean()
    
    macdo = macd - macds 
    
    df = df.assign(macd=macd, macds=macds, macdo=macdo).dropna()
    
    return df

In [17]:
df = get_macd(df)

df.head()

,date,close,diff,open,high,low,volume,kdj_d,kdj_j,kdj_k,macd,macdo,macds
298,2018-08-27,66300,1300,66200,67500,64700,1302620,57.142857,57.142857,57.142857,0.000000,0.000000,0.000000
297,2018-08-28,67400,1100,67300,69100,66500,1677504,59.675325,58.831169,61.363636,24.679487,10.968661,13.710826
296,2018-08-29,66100,1300,67900,68100,65300,1044419,46.479836,51.773264,31.818182,-8.866357,-13.324239,4.457882
295,2018-08-30,68500,2400,65800,68600,65400,1002923,63.046953,57.785898,86.363636,62.744713,38.541970,24.202744
294,2018-08-31,76400,7900,67900,79100,67700,5987954,70.034853,64.107939,81.250000,414.695345,274.329881,140.365464


In [19]:
import plotly.offline as offline 
import plotly.graph_objs as go
from plotly import tools
 
# jupyter notebook 에서 출력 
offline.init_notebook_mode(connected=True)
 
kdj_k = go.Scatter(
                x=df.date,
                y=df['kdj_k'],
                name="Fast%K")
 
kdj_d = go.Scatter(
                x=df.date,
                y=df['kdj_d'],
                name="Fast%D")
 
kdj_d2 = go.Scatter(
                x=df.date,
                y=df['kdj_d'],
                name="Slow%K")
 
kdj_j = go.Scatter(
                x=df.date,
                y=df['kdj_j'],
                name="Slow%D")

macd = go.Scatter(
    x=df.date,
    y=df['macd'],
    name = "MACD"
)

signal = go.Scatter(
    x=df.date,
    y=df['macds'],
    name = "Signal"
)

oscillator = go.Scatter(
    x=df.date,
    y=df['macdo'],
    name = "oscillator"
)

trade_volume = go.Scatter(
    x=df.date,
    y=df['volume'],
    name = "volume"
)

 
trade_volume = go.Bar(
                x=df.date,
                y=df['volume'],
                name="volume")
 
 
# data = [kdj_k, kdj_d]
data1 = [kdj_d2, kdj_j,macd,signal,oscillator]
data2 = [trade_volume]
 
# data = [celltrion]
# layout = go.Layout(yaxis=dict(
#         autotick=False,
#         ticks='outside',
#         tick0=0,
#         dtick=10,
#         ticklen=8,
#         tickwidth=4,
#         tickcolor='#000'
#     ))
 
fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True)
 
for trace in data1:
    fig.append_trace(trace, 1,1)
 
for trace in data2:
    fig.append_trace(trace, 2,1)
# fig = go.Figure(data=data, layout=layout)
 
offline.iplot(fig)


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

